# Heat equation on a time-dependent domain: Example 5

With the knowledge gained from the previous examples, we can now solve the drilling problem. Here, we consider
the heat equation on a time-dependent domain $\Omega(t)$. For the PDE we consider, for $t\in[0, 5]$,
\begin{align*}
    \partial_t u(x, t) -0.1\Delta u(x, t) &= 0, && \text{ in }  \Omega(t), \\
    u(x, 0) &= 0 , && \text{ in }  \Omega(0), \\
    u(x, t) &= 0 , &&\text{ for } x_2=0, \\
    0.1 \nabla u(x, t)\cdot n &= 0 , &&\text{ on } \Gamma_N(t), \\
    0.1\nabla u(x, t)\cdot n &= f , &&\text{ on } \Gamma_H(t).
\end{align*}

Now let $D(t)$ be the drill that we use for removing material from our original domain $\Omega(0)=[0, 1] \times [0, 1]$. Here, we consider a rectangle $D(t) = [0.2, 0.4] \times [h(t), 1.0]$, where $h(t)$ describes the depht of the drill at point $t$ and is given by
\begin{align*}
    h(t) = 1.0 - v_\text{drill} \cdot t.
\end{align*}
The speed of removing the material is given by $v_\text{drill}$.

Then our time dependent domain is given by $\Omega(t) = ([0, 1] \times [0, 1]) \setminus D(t)$.

The boundary parts are given by
\begin{align*}
    \Gamma_H(t) &=  [0.2, 0.4] \times \{h(t)\}, \\
    \Gamma_N(t) &=  \partial ([0, 1] \times [0, 1]) \setminus (\{x_2=0\} \cup \Gamma_H(t)),
\end{align*}
at the bottom boundary ($x_2=0$) we have a fixed temperature $u=0$, at the bottom of the drill part ($\Gamma_H$) a heat source and 
everywhere else ($\Gamma_N$) homogeneous Neumann conditions.

For visualization and to better understand the geometry, we prepared a simple depcition of the domain:

In [2]:
# Just run the cell to see the domain
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/TomF98/torchphysics/Berlin-Workshop/examples/SolutionData/DrillingData/drill_domain.png")

In [ ]:
# This block is for GPU selection. Please execute.
import pathlib
import os
user = int(str(pathlib.Path().resolve())[-2:])
if user <= 21: 
    gpu_device = str(user % 7) # moriarty
else: gpu_device = str(user % 4) # neptuno
os.environ["CUDA_VISIBLE_DEVICES"]= gpu_device

In [1]:
import torch
import torchphysics as tp
import pytorch_lightning as pl
import math 

# Here all parameters are defined:
t_min, t_max = 0.0, 5.0
t_prod_end = 5.0
prod_speed = 1.0 # speed of heating 

size_x, size_y = 1.0, 1.0
x_start, x_end = 0.2, 0.4 # x-size of drill hole
y_end = 0.4 # end height of drill hole

drill_speed = (size_y - y_end) / (t_prod_end)

kappa = 0.1 # heat diffusion

# Function for heat source
def f(t, x):
    heat_source = 80 * (x[:, :1] - x_start) * (x_end - x[:, :1])
    heat_source *= (1.0 - torch.exp(-prod_speed*t))
    return heat_source

# Movement of drill (returns the x_2 position of the bottom of the drill head)
def drill_depth(t):
    height = size_y - drill_speed * t
    return height

In [2]:
# First we have to implement the spaces that appear:
X = tp.spaces.R2('x')
T = tp.spaces.R1('t')
U = tp.spaces.R1('u')

Now, we handle the time dependent domain. The basic domain creation is like we have seen before.
Special here is now, that we use Pyhton-functions to describe the corners of our drill hole.

To create our domain, we define first the square $[0, 1] \times [0, 1]$ and then substract the drill hole (a 
time dependent square) from it.

In [3]:
# TODO: Implement the unit square, like below:
#       tp.domains.Parallelogram(Space, lower_left_corner, lower_right_corner, upper_left_corner)
#       Order of corners is important!
box = ...

# This is the lower left corner of our drill hole. At a given input t we have to return 
# the correct coordinate of the lower left corner.
def left_corner(t):
    # First construct a place holder tensor, where we can input the correct coordinates.
    coordinate = torch.zeros((len(t), 2), device=t.device)
    # The x_1-coordinate is fixed, independent of t:
    coordinate[:, :1] = x_start
    # The x_2-coordinate is:
    coordinate[:, 1:2] = drill_depth(t)
    return coordinate

# TODO: Implement the lower right corner:
def right_corner(t):
    pass

# Now we can use the above functions as input arguments in the Parallelogram.
# The top left corner is fixed outside the unit square (for numerical reasons), such that we obtain  
# a rectangle that grows in the negative x_2 direction over time.
drill_hole = tp.domains.Parallelogram(X, left_corner, right_corner, [x_start, size_y+0.1])

# TODO: Construct the difference of the "box" and "drill_hole"
omega = ...

# TODO: Implement the time interval
I_t = ...

The defined domain $\Omega$ is now dependent on time, since the *drill_hole* depends on time.
This we have to keep in mind when we want to create points inside this domain. To sample points,
we have to say what time point $t$ we are considering.

This order is also important for the *Samplers* in TorchPhysics. If we input the "Cartesian" Product *omega \* I_t*, the sampler will create points from right to left, first points in the time interval and then pass them to the space domain to create valid points. 

The combination *I_t \* omega* does **not** work. 

In [ ]:
# TODO: Experiment a bit with different Samplers and number of points to get a feeling of the
#       time dependent domain.

inner_sampler = tp.samplers.RandomUniformSampler(omega * I_t, 5000)
bc_sampler = tp.samplers.RandomUniformSampler(box.boundary*I_t, 2000)

# One can also multiply (create the Cartesian product) of the samplers, which allows for 
# grid sampling. 
time_sampler = tp.samplers.GridSampler(I_t, 10)
drill_sampler = tp.samplers.RandomUniformSampler(drill_hole.boundary, 50) * time_sampler

# With scatter we can visualize one example batch of points.
# First argument: The space over which we want to plot (here either: X, T or X*T)
# Later arguments all sampler we want to visualize.
# Note: The points in the plot not always have the correct three-dimensional "depth", 
# they may incorrectly appear behind or in front of each other.
fig = tp.utils.scatter(X, drill_sampler, bc_sampler)

For training we use the following samplers:

In [ ]:
# When making a sampler static (e.g. fixing the points it creates) we can also pass specify
# after how many training iterations new points will be created.
# We do this here to speed up the training process, since sampling in time dependent domains
# is a little bit slower than in normal domains. 
inner_sampler = tp.samplers.RandomUniformSampler(omega * I_t, 75000).make_static(resample_interval=250)
bc_sampler = tp.samplers.RandomUniformSampler(box.boundary*I_t, 25000)

time_sampler = tp.samplers.GridSampler(I_t, 120).make_static()
drill_sampler = tp.samplers.RandomUniformSampler(drill_hole.boundary, 2000) * time_sampler

In [ ]:
# Next the neural network:
model = tp.models.FCN(input_space=X*T, output_space=U, 
                      hidden=(80, 80, 80, 80, 80, 80, 80))

To simplify the training procedure, we implement some conditions into the network architecture by using the so-called *hard constrains*.
These include the initial condition $u(\cdot, 0) = 0$ and the Dirichlet condition at the lower boundary. 

A simple way to achieve this is to multiply the output of the neural network by some factors such that the conditions are fulfilled.

In our case this is possible with
\begin{align*}
    x_2 \cdot u \cdot (1.0 - e^{-t}),
\end{align*}
where $u$ is the output of the neural network.

In [ ]:
# TODO: Implement the hard constrains
# Hint: Use torch.exp
def constrain_fn(u, x, t):
    return x[:, 1:2] * ...

Next, we implement the PDE via the *PINNCondition* as we did before.

In [113]:
# Start with the heat equation
def pde_residual(u, x, t):
    u = constrain_fn(u, x, t) # here remember to apply our constrain 
    return kappa * tp.utils.laplacian(u, x) - tp.utils.grad(u, t)

pde_condition = tp.conditions.PINNCondition(module=model,
                                            sampler=inner_sampler,
                                            residual_fn=pde_residual,
                                            name='pde_condition', weight=50)

In [ ]:
# Next the boundary conditions at the drill hole.
# It looks a bit complicated, therefore we implemented it for you.
tol = 0.0001 # small tolerance for filtering points

def drill_residual(u, x, t):
    # Apply hard constrains:
    u = constrain_fn(u, x, t)

    resiudal = torch.zeros_like(u) # a tensor to write the residual into
    normal_vectors = -drill_hole.boundary.normal(x, {"t": t})
    u_n = tp.utils.normal_derivative(u, normal_vectors, x)
    
    # Evaluate heat source everywhere (filter out correct points later)
    heat_source = f(t, x) 
    # The upper boundary of our drill hole rectangle is not inside the domain
    # -> filter out the correct points
    point_in_domain = (x[:, 1:] <= size_y)

    # Homogeneous Neumann condition on drill hole
    resiudal[point_in_domain] = u_n[point_in_domain]
    
    # On the bottom side on the hole we have a heat source.
    # First filter the points: 
    drill_section_x = torch.logical_and(x[:, :1] > x_start + tol, x[:, :1] < x_end - tol)
    drill_section = torch.logical_and(point_in_domain, drill_section_x)
    # Then construct residual:
    resiudal[drill_section] = (u_n - heat_source/kappa)[drill_section]

    return resiudal

drill_condition = tp.conditions.PINNCondition(module=model,
                                           sampler=drill_sampler,
                                           residual_fn=drill_residual,
                                           name='drill_condition', weight=2)

In [115]:
# TODO: Implement the condition for the outer boundary (that is not on the drill hole).
#       This resiudal will automatically only be applied on points on the boundary of the unit 
#       square (0, 1) x (0, 1).

def bc_residual(u, x, t):
    # Apply hard constrains:
    
    # Compute normal vectors and derivative:

    # Construct residual for the Neumann condition:

    return 

bc_condition = tp.conditions.PINNCondition(module=model,
                                           sampler=bc_sampler,
                                           residual_fn=bc_residual,
                                           name='bc_condition', weight=100)

Now, we can start the training of the neural network. 
In our testing we found that first only training the boundary condition on the drill part is helpful:

In [ ]:
optim = tp.OptimizerSetting(optimizer_class=torch.optim.Adam, lr=1.e-4)
solver = tp.solver.Solver([drill_condition], optimizer_setting=optim)

trainer = pl.Trainer(devices=1, accelerator="gpu",
                     num_sanity_val_steps=0,
                     benchmark=True,
                     max_steps=2500, 
                     logger=False, 
                     enable_checkpointing=False
                     )

trainer.fit(solver)

Next, we combine all loss terms in the training process. Here, we only train for a short time frame. Because we need a rather large network and the GPUs on Google Colab are not that fast, so that the training is slow. 

But we can still obtain a good first approximation and in the later cells also load a pretrained network, that we trained for roughly 25 minutes on a GTX 2080 beforehand.

In [ ]:
optim = tp.OptimizerSetting(optimizer_class=torch.optim.Adam, lr=1.e-3) 
solver = tp.solver.Solver([pde_condition, drill_condition, bc_condition],
                        optimizer_setting=optim)

trainer = pl.Trainer(devices=1, accelerator="gpu",
                    num_sanity_val_steps=0,
                    benchmark=True,
                    max_steps=1500,
                    logger=False, 
                    enable_checkpointing=False
                    )

trainer.fit(solver)

First we can have a look on the learned solution:

In [ ]:
anim_sampler = tp.samplers.AnimationSampler(omega, I_t, 100, n_points=1000)
fig, anim = tp.utils.animate(model, lambda u, t, x: constrain_fn(u, x, t), 
                             anim_sampler, ani_speed=10, ani_type='contour_surface')
anim.save('moving-heat-eq.gif')
# On Google colab you have at the left side a tab with a folder. There you can find the gif and can watch it.

We also prepared a FEM-solution for comparision. First, we download the data from GitHub and then read it with
numpy.

In [ ]:
!wget https://github.com/TomF98/torchphysics/raw/Berlin-Workshop/examples/SolutionData/DrillingData/time_points.npy
!wget https://github.com/TomF98/torchphysics/raw/Berlin-Workshop/examples/SolutionData/DrillingData/coordinates.npy
!wget https://github.com/TomF98/torchphysics/raw/Berlin-Workshop/examples/SolutionData/DrillingData/solution.npy

In [ ]:
import numpy as np

time_points = np.load("time_points.npy")
coords = np.load("coordinates.npy")
fem_sol = np.load("solution.npy")

In [ ]:
# Now evaluate the PINN solution at the FEM-points and compare the solution.
pinn_sol = torch.zeros((len(time_points), len(coords[0])))

for i in range(len(time_points)):
    input_points = torch.zeros((len(coords[i]), 3))
    input_points[:, :2] = torch.tensor(coords[i], dtype=torch.float32)
    input_points[:, 2] = time_points[i]
    model_out = model(tp.spaces.Points(input_points, X*T)).as_tensor
    pinn_sol[i, :] = constrain_fn(model_out, input_points[:, :2], input_points[:, 2:]).flatten()

print("Difference to FEM in Sup-norm:")
difference_sup = np.max(np.abs(fem_sol - pinn_sol.detach().numpy()))
print("Absolute:", difference_sup)
print("Relative:", difference_sup / np.max(fem_sol))

Since we only trained for a short time frame, the relative error is still large.
Therefore, we also prepared a neural network that has been trained a bit longer.

In [ ]:
# Get the network from Github
!wget https://github.com/TomF98/torchphysics/raw/Berlin-Workshop/examples/SolutionData/DrillingData/model_drilling.pt

In [175]:
# The following lines show how one can save and load a neural network
torch.save(model.state_dict(), 'model_drilling_short_training.pt')

model.load_state_dict(torch.load('model_drilling.pt'))

<All keys matched successfully>

We evaluate the new model and obtain a relative error of around 9%.

In [176]:
pinn_sol = torch.zeros((len(time_points), len(coords[0])))

for i in range(len(time_points)):
    input_points = torch.zeros((len(coords[i]), 3))
    input_points[:, :2] = torch.tensor(coords[i], dtype=torch.float32)
    input_points[:, 2] = time_points[i]
    model_out = model(tp.spaces.Points(input_points, X*T)).as_tensor
    pinn_sol[i, :] = constrain_fn(model_out, input_points[:, :2], input_points[:, 2:]).flatten()

In [ ]:
print("Difference to FEM in Sup-norm:")
difference_sup = np.max(np.abs(fem_sol - pinn_sol.detach().numpy()))
print("Absolute:", difference_sup)
print("Relative:", difference_sup / np.max(fem_sol))